# Gurobi optimization using one machine learning model
## Optimize for Price and Supply of Avocados

- In this example is only linear regressions, but gurobi machine learning acept multiple models. Documentation **"gurobi-machinelearning"**

https://gurobi-machinelearning.readthedocs.io/en/stable/api.html


- In addition, to define the decision variables, parameters, restriction, etc of the optimization model are used **"gurobipy-pandas"**. Using this package is possible define the optimization model using pandas DataFrames

https://gurobipy-pandas.readthedocs.io/en/stable/

In [1]:
import pickle
import pandas as pd
import numpy as np

#gurobi
import gurobipy_pandas as gppd
from gurobi_ml import add_predictor_constr
import gurobipy as gp

### 0. Root repo

In [2]:
import os
# fix root path to save outputs
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('\\')[:-1]
root_path = '\\'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

root path:  D:\github-mi-repo\Examples-Gurobi-ML


## PREPARATION

### 1. Load data needs to use
In this example data is loaded because it is necesary to generate parameters of optimization model

In [3]:
# read data that have all the units sold for each region
path_data_basic_features = 'artifacts/data/data_basic_features.pkl'
data_units_sold = pd.read_pickle(path_data_basic_features)

In [4]:
##### use data to generate parameters for optimization model

# min, max deliry each region
data_min_delivery = data_units_sold.groupby("region")["units_sold"].min().rename('min_delivery')
data_max_delivery = data_units_sold.groupby("region")["units_sold"].max().rename('max_delivery')

# historical distribution of price each region
data_historical_max_price = data_units_sold.groupby("region")["price"].max().rename('max_price')

### 2. Load model machine learning
Load model that given an input (price and other features) predict the price.

The model was trained in the notebook "models/1_basic_features_one_lr"

In [5]:
# folder
path_folder_artifacts = 'artifacts/models/1_basic_features_one_lr/'

# path to save
name_artifact_model = 'model.pkl'
path_artifact_model = path_folder_artifacts + name_artifact_model

# load model
with open(path_artifact_model, 'rb') as artifact:
    model = pickle.load(artifact)

model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='first'),
                                                  ['region']),
                                                 ('standardscaler',
                                                  StandardScaler(), ['price']),
                                                 ('passthrough', 'passthrough',
                                                  ['peak'])],
                                   verbose_feature_names_out=False)),
                ('linearregression', LinearRegression())])

## RUN OPTIMIZATION

### 0. Load transversal params - sets of optimization model
Transversal all codes, not only this code. For example order in features in the data.

Save the sets of optimization model as pandas index

In [6]:
list_regions = ['Great_Lakes',
                'Midsouth',
                'Northeast',
                'Northern_New_England',
                'Plains',
                'SouthCentral',
                'Southeast',
                'West']
regions = list_regions

regions

['Great_Lakes',
 'Midsouth',
 'Northeast',
 'Northern_New_England',
 'Plains',
 'SouthCentral',
 'Southeast',
 'West']

In [7]:
# generate a pandas index with the values of the regions. This works as sets of optimization model
index_regions = pd.Index(regions)

### 1. Create guroby optimization model

In [8]:
# env = gp.Env(params=params)

#Create the model within the Gurobi environment
m = gp.Model(name = "Avocado_Price_Allocation")

Restricted license - for non-production use only - expires 2025-11-24


### 2. Upper bounds and lower bounds of decision variables
Values that are boundss in decision variables. In gurobi the upper and lower boundss could be defined in the same moment that variables are created and not are defined as restrictions explicitly 

- $a_{min},a_{max}$: minimum and maximum price ($\$$) per avocado (price is a input of machine learning model)
- $b^r_{min},b^r_{max}$: minimum and maximum number of avocados allocated to region $r$

In [9]:
# a_min, a_max: min and max price of product A
a_min = 0
a_max = 2


# b_min(r), b_max(r): min and max historical products send to each region (value get from historical data)
b_min = data_min_delivery
b_max = data_max_delivery

### 3. Input parameters of optimization model
##### That are not decision variables either parameters of machine learning model)

**Set**
- $r$ : will be used to denote each region


**Parameters Optimization Model**
- $B$: available avocados to be distributed across the regions.Total amount of avocado supply

- $c_{waste}$: cost ($\$$) per wasted avocado

- $c^r_{transport}$: cost ($\$$) of transporting a avocado to region $r$

In [10]:
# B: supply product
B = 30 


# c_waste: cost of waste product
c_waste = 0.1


# c_transport(r): cost transport for each region
c_transport = pd.Series(
    {
        "Great_Lakes": 0.3,
        "Midsouth": 0.1,
        "Northeast": 0.4,
        "Northern_New_England": 0.5,
        "SouthCentral": 0.3,
        "Southeast": 0.2,
        "West": 0.2,
        "Plains": 0.2,
    }, name='transport_cost')
c_transport = c_transport.loc[regions]

### 4. Features input machine learning model fixed (that are not decision variables or parameters in optimization model)
Define the features that are inputs of machine learning model that are not decision variables of optimization model (so this values doesn't change). And also, this features that are not parameters of optimization model, so this values are not used in the restrictions

In [11]:
# seasonality: 1 if it is the peak season; 0 if isn't
peak_or_not = 0
peak_or_not

0

In [12]:
# list of regions
regions # in this example the regions is also a feature of machine learning model - and we want to know the price for each region(r)

['Great_Lakes',
 'Midsouth',
 'Northeast',
 'Northern_New_England',
 'Plains',
 'SouthCentral',
 'Southeast',
 'West']

In [13]:
# generate a dataframe with the "fixed" features of optimization model. 
# This is an instance of machine learning model. In this part only have the features that have fixed values for this optimization
instance_ml_model = pd.DataFrame(
    data={
        "peak": peak_or_not,
        "region": regions,
    },
    index=regions
)
instance_ml_model

,peak,region
Great_Lakes,0,Great_Lakes
Midsouth,0,Midsouth
Northeast,0,Northeast
Northern_New_England,0,Northern_New_England
Plains,0,Plains
SouthCentral,0,SouthCentral
Southeast,0,Southeast
West,0,West


### 5. Decision variables of optimization model

Let us now define the decision variables. In our model, we want to store the price and number of avocados allocated to each region. We also want variables that track how many avocados are predicted to be sold and how many are predicted to be wasted. 

- $p(r)$ the price of an avocado ($\$$) in each region. The maxium price. It is a feature of machine learning model
- $x(r)$ the number of avocados supplied to each region
- $s(r)$ the predicted number of avocados sold in each region
- $u(r)$ the predicted number of avocados unsold (wasted) in each region
- $d(r)$ the predicted demand in each region. It is the target of machine learning model (because this value change according the input, it is a decision variable)

All those variables are created using gurobipy-pandas, with the function `gppd.add_vars`. To use this function it is necessary to define:
- model: optimization model of gurobi
- index: pandas index. With this index it can defined the sets of the decision variables
- name: name of the decision variable
- Example: x = gppd.add_vars(model, index, name="x")

In [14]:
# p(r): price. feature of machine learning model
p = gppd.add_vars(m, index_regions, name = "price", lb = a_min, ub = a_max) # bounds prices


# x(r): supply
x = gppd.add_vars(m, index_regions, name = "x", lb = b_min, ub= b_max) # bounds supply - using historical data


# s(r): solds given a certain price
s = gppd.add_vars(m, index_regions, name = "s")


# u(r): inventary. units not sold. waste.
u = gppd.add_vars(m, index_regions, name = "w") 


# d(r): demand. output of machine learning model
d = gppd.add_vars(m, index_regions, lb = -gp.GRB.INFINITY, name = "demand") # BY DEFULT LOWER BOUND IS ZERO



### 6. Constraints (constraints that are not generated by a ml model)

#### 6.1 Add the Supply Constraint
Make sure that the total number of avocados supplied is equal to $B$
\begin{align*} \sum_{r} x_r &= B \end{align*}

In [15]:
# SAVE THE CONSTRAINT IN A PYTHON VARIABLE
supply_constraint = m.addConstr(x.sum() == B)

In [16]:
supply_constraint # OBS IN THIS EXAMPLE THE CONSTRAINT IS SAVED IN PYTHON VARIABLE - it is not neccesary - only for debugging it is a little useful

<gurobi.Constr *Awaiting Model Update*>

In [17]:
# see constraint
m.update()
m.getRow(supply_constraint)

<gurobi.LinExpr: x[Great_Lakes] + x[Midsouth] + x[Northeast] + x[Northern_New_England] + x[Plains] + x[SouthCentral] + x[Southeast] + x[West]>

#### 6.2 Add Constraints That Define Sales Quantity
The sales quantity is the minimum of the allocated quantity and the predicted demand, i.e., $s_r = \min \{x_r,d_r(p_r)\}$ This relationship can be modeled by the following two constraints for each region $r$.

\begin{align*} s_r &\leq x_r                \:\:\:\:\forall r\\
s_r &\leq d(p_r,r)                   \:\:\:\:\forall r\end{align*}

In [18]:
gppd.add_constrs(m, s, gp.GRB.LESS_EQUAL, x) # for each region (8 constraints)
gppd.add_constrs(m, s, gp.GRB.LESS_EQUAL, d)
# add_constrs: write the constraint with left side, "operator", right side

Great_Lakes             <gurobi.Constr *Awaiting Model Update*>
Midsouth                <gurobi.Constr *Awaiting Model Update*>
Northeast               <gurobi.Constr *Awaiting Model Update*>
Northern_New_England    <gurobi.Constr *Awaiting Model Update*>
Plains                  <gurobi.Constr *Awaiting Model Update*>
SouthCentral            <gurobi.Constr *Awaiting Model Update*>
Southeast               <gurobi.Constr *Awaiting Model Update*>
West                    <gurobi.Constr *Awaiting Model Update*>
dtype: object

#### 6.3 Add the Wastage Constraints
Define the predicted unsold number of avocados in each region, given by the supplied quantity that is not sold. For each region $r$.

\begin{align*} u_r &= x_r - s_r                 \:\:\:\:\forall r\end{align*}

In [19]:
gppd.add_constrs(m, u, gp.GRB.EQUAL, x - s, name = 'waste_constraint') # OBS IN THIS EXAMPLE THE CONSTRAINT IS DEFINED WITH A NAME - it is useful

Great_Lakes             <gurobi.Constr *Awaiting Model Update*>
Midsouth                <gurobi.Constr *Awaiting Model Update*>
Northeast               <gurobi.Constr *Awaiting Model Update*>
Northern_New_England    <gurobi.Constr *Awaiting Model Update*>
Plains                  <gurobi.Constr *Awaiting Model Update*>
SouthCentral            <gurobi.Constr *Awaiting Model Update*>
Southeast               <gurobi.Constr *Awaiting Model Update*>
West                    <gurobi.Constr *Awaiting Model Update*>
Name: waste_constraint, dtype: object

#### 6.4 Model update - add the constraint to gurobi model

In [20]:
m.update()

In [21]:
### show all decision variables - debugging problems - validate after compile decision varaibles
#p.gppd.VarName  # see name
p.gppd.ub # upper bound
#p.gppd.lb # lowe bound

#p.gbpd.X # see value of decision variable - only works after optimization

Great_Lakes             2.0
Midsouth                2.0
Northeast               2.0
Northern_New_England    2.0
Plains                  2.0
SouthCentral            2.0
Southeast               2.0
West                    2.0
Name: price, dtype: float64

In [22]:
m

<gurobi.Model Continuous instance Avocado_Price_Allocation: 25 constrs, 40 vars, No parameter changes>

### 7. Add constraints that are machine learning models

#### 7.1 predict demand - generate instance dataframe

First, we create our full input for the predictor constraint. We concatenate the `p` variables and the fixed features

In [23]:
## generate a pandas instance for machine learning model

m_instance_ml_model = pd.concat([instance_ml_model, p], axis=1) # generate instance for optimization model
list_features = ['region', 'peak', 'price'] # list of the order of the features used to train the ml model
m_instance_ml_model = m_instance_ml_model[list_features] # sort instance with the correct order
m_instance_ml_model

,region,peak,price
Great_Lakes,Great_Lakes,0,<gurobi.Var price[Great_Lakes]>
Midsouth,Midsouth,0,<gurobi.Var price[Midsouth]>
Northeast,Northeast,0,<gurobi.Var price[Northeast]>
Northern_New_England,Northern_New_England,0,<gurobi.Var price[Northern_New_England]>
Plains,Plains,0,<gurobi.Var price[Plains]>
SouthCentral,SouthCentral,0,<gurobi.Var price[SouthCentral]>
Southeast,Southeast,0,<gurobi.Var price[Southeast]>
West,West,0,<gurobi.Var price[West]>


#### 7.2 predict demand - generate machine learning constraint
Call
[add_predictor_constr](https://gurobi-machinelearning.readthedocs.io/en/stable/auto_generated/gurobi_ml.add_predictor_constr.html)
to insert the constraints linking the features and the demand into the model `m`.

It is important that you keep the columns in the order above, otherwise you will see an error. The columns must be in the same order as the training data.

Obs: to add this constraints the way is little different and it is not neccesary call model.update()

**Documentation - parameters**

- gp_model (gurobipy model) – The gurobipy model where the predictor should be inserted.

- predictor – The predictor to insert.

- input_vars (mvar_array_like) – Decision variables used as input for predictor in gp_model.

- output_vars (mvar_array_like, optional) – Decision variables used as output for predictor in gp_model.

In [24]:
## add model to predict the demand for each region with the SAME MODEL
pred_constr = add_predictor_constr(gp_model = m, 
                                   predictor = model, 
                                   input_vars = m_instance_ml_model, 
                                   output_vars = d
                                  )

In [25]:
pred_constr.print_stats()

Model for pipe:
72 variables
16 constraints
Input has shape (8, 3)
Output has shape (8, 1)

Pipeline has 2 steps:

--------------------------------------------------------------------------------
Step            Output Shape    Variables              Constraints              
                                                Linear    Quadratic      General
col_trans             (8, 9)            8            8            0            0

lin_reg               (8, 1)           64            8            0            0

--------------------------------------------------------------------------------


In [26]:
m

<gurobi.Model Continuous instance Avocado_Price_Allocation: 41 constrs, 112 vars, No parameter changes>

### 8. Define Objetive Function
The goal is to maximize the **net revenue**, which is the product of price and quantity, minus costs over all regions. This model assumes the purchase costs are fixed (since the amount $B$ is fixed) and are therefore not incorporated.

\begin{align} \textrm{maximize} &  \sum_{r}  (p_r * s_r - c_{waste} * u_r -
c^r_{transport} * x_r)& \end{align}

In [27]:
m.setObjective((p * s).sum() - c_waste * u.sum() - (c_transport * x).sum(),
               gp.GRB.MAXIMIZE)

### 9. Solve optimization problem
The objective is **quadratic** since we take the product of price and the predicted sales, both of which are variables. Maximizing a quadratic
term is said to be **non-convex**, and we specify this by setting the value of the [Gurobi NonConvex
parameter](https://www.gurobi.com/documentation/10.0/refman/nonconvex.html) to be $2$.

#### 9.1 Solve optimization problem

In [28]:
# solve cuadratic problems
m.Params.NonConvex = 2

Set parameter NonConvex to value 2


In [29]:
# solve
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19043.2))

CPU model: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 41 rows, 112 columns and 160 nonzeros
Model fingerprint: 0xd98757a0
Model has 8 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-01, 3e+00]
  Objective range  [1e-01, 5e-01]
  QObjective range [2e+00, 2e+00]
  Bounds range     [2e-01, 1e+01]
  RHS range        [1e+00, 3e+01]
Presolve removed 16 rows and 80 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 24 rows and 88 columns
Presolve time: 0.00s
Presolved: 34 rows, 33 columns, 81 nonzeros
Presolved model has 8 bilinear constraint(s)
Variable types: 33 continuous, 0 integer (0 binary)
Found heuristic solution: objective 29.2230179

Root relaxation: objective 4.765570e+01, 36 iterations, 0.00 seconds (0.00 work units)


In [30]:
#### know the status of the model - 2 a optimal solution was founded
# docu: https://www.gurobi.com/documentation/current/refman/optimization_status_codes.html#sec:StatusCodes
m.Status

2

#### 9.2 Save optimal values in a dataframe
To get the optimal values of decision variables it is neccesary call "var.gppd.X"

In [31]:
# create dataframe with index
solution = pd.DataFrame(index=index_regions)

# save optimal values
solution["Price"] = p.gppd.X
solution["Historical_Max_Price"] = data_historical_max_price  # this is informative value get from historical data
solution["Allocated"] = x.gppd.X
solution["Sold"] = s.gppd.X
solution["Wasted"] = u.gppd.X
solution["Pred_demand"] = d.gppd.X

# round values
solution = solution.round(3)


# get value objetive function
opt_revenue = m.ObjVal

In [32]:
# show value objetive function
print("\n The optimal net revenue: $%f million" % opt_revenue)


 The optimal net revenue: $29.223018 million


In [33]:
# show value decision variables
solution

,Price,Historical_Max_Price,Allocated,Sold,Wasted,Pred_demand
Great_Lakes,1.400,1.980,2.887,2.887,0.00,2.887
Midsouth,1.277,1.720,6.169,2.839,3.33,2.839
Northeast,1.799,1.750,3.504,3.504,0.00,3.504
Northern_New_England,0.885,1.880,0.918,0.918,0.00,0.918
Plains,0.905,1.810,2.713,2.073,0.64,2.073
SouthCentral,1.822,1.366,3.757,3.757,0.00,3.757
Southeast,1.496,1.820,4.451,3.291,1.16,3.291
West,1.977,1.648,5.603,4.283,1.32,4.283


We can also check the error in the estimate of the Gurobi solution for the regression model.

In [34]:
print(
    "Maximum error in approximating the regression {:.6}".format(
        np.max(pred_constr.get_error())
    )
)

Maximum error in approximating the regression 2.22045e-15


### 9.3 Show values of constraints

DOCUMENTAITON: ATRIBUTES EACH CLASS GUROBI: https://www.gurobi.com/documentation/current/refman/attributes.html#sec:Attributes

#### 9.3.1 show constraint defined as python variable
When a constraint is defined and added to optimization model, if the constraint is saved in a python variable, it is possible to see the value of an specif constraint

In [35]:
# show constraint after optimization
supply_constraint

<gurobi.Constr R0>

In [36]:
supply_constraint.ConstrName

'R0'

In [37]:
#### KNOW LEFT SIDE CONSTRAINT: https://support.gurobi.com/hc/en-us/articles/9424726080529-How-do-I-access-the-left-hand-side-of-a-constraint
m.getRow(supply_constraint)

<gurobi.LinExpr: x[Great_Lakes] + x[Midsouth] + x[Northeast] + x[Northern_New_England] + x[Plains] + x[SouthCentral] + x[Southeast] + x[West]>

In [38]:
# right side constraint
supply_constraint.RHS

30.0

In [39]:
# inequation constraint
supply_constraint.Sense

'='

#### 9.3.2 Show all constraints

In [40]:
m.getConstrs()

[<gurobi.Constr R0>,
 <gurobi.Constr R1>,
 <gurobi.Constr R2>,
 <gurobi.Constr R3>,
 <gurobi.Constr R4>,
 <gurobi.Constr R5>,
 <gurobi.Constr R6>,
 <gurobi.Constr R7>,
 <gurobi.Constr R8>,
 <gurobi.Constr R9>,
 <gurobi.Constr R10>,
 <gurobi.Constr R11>,
 <gurobi.Constr R12>,
 <gurobi.Constr R13>,
 <gurobi.Constr R14>,
 <gurobi.Constr R15>,
 <gurobi.Constr R16>,
 <gurobi.Constr waste_constraint[Great_Lakes]>,
 <gurobi.Constr waste_constraint[Midsouth]>,
 <gurobi.Constr waste_constraint[Northeast]>,
 <gurobi.Constr waste_constraint[Northern_New_England]>,
 <gurobi.Constr waste_constraint[Plains]>,
 <gurobi.Constr waste_constraint[SouthCentral]>,
 <gurobi.Constr waste_constraint[Southeast]>,
 <gurobi.Constr waste_constraint[West]>,
 <gurobi.Constr pipe.col_trans.std_scaler.s[0,0]>,
 <gurobi.Constr pipe.col_trans.std_scaler.s[1,0]>,
 <gurobi.Constr pipe.col_trans.std_scaler.s[2,0]>,
 <gurobi.Constr pipe.col_trans.std_scaler.s[3,0]>,
 <gurobi.Constr pipe.col_trans.std_scaler.s[4,0]>,
 <guro

In [41]:
m.getConstrs()[0]

<gurobi.Constr R0>

#### 9.3.3 show constraint by name
Get atributes of constraint by the name of the constraint

In [42]:
# define a name of constraint to get its values
constraint_example_name = 'waste_constraint[Midsouth]' # the name of constraint and the region

In [43]:
# get an object gurobi.constr searching by name
constraint_example = m.getConstrByName(constraint_example_name)

In [44]:
# left side
m.getRow(constraint_example)

<gurobi.LinExpr: -1.0 x[Midsouth] + s[Midsouth] + w[Midsouth]>

In [45]:
# right side
constraint_example.RHS

0.0

In [46]:
# inequation
m.getConstrByName('waste_constraint[Midsouth]').Sense

'='

In [47]:
### example model constraint

In [48]:
m.getRow(m.getConstrByName('pipe.lin_reg.linreg[0,0]'))

<gurobi.LinExpr: demand[Great_Lakes] + 0.4482128990683356 pipe.col_trans.std_scaler.out[0,0] + 0.30302380204935636 pipe.lin_reg.feat0[0] + -1.4399463873214375 pipe.lin_reg.feat1[0] + 3.030572296572469 pipe.lin_reg.feat2[0] + 1.834754987011753 pipe.lin_reg.feat3[0] + -1.7398379538907354 pipe.lin_reg.feat4[0] + -0.6018601658254568 pipe.lin_reg.feat5[0] + -2.5858079728301853 pipe.lin_reg.feat6[0] + -0.60240802686842 pipe.lin_reg.feat8[0]>

In [49]:
m.getConstrByName('pipe.lin_reg.linreg[0,0]').Sense

'='

In [50]:
m.getConstrByName('pipe.lin_reg.linreg[0,0]').RHS

3.4167831308090957

In [51]:
m.getConstrByName('pipe.lin_reg.linreg[0,0]').Slack

0.0

## Important question
If it is a real use case, and I want to train different models more complex that the previous models, how I can to know the impact to have a better model (better metrics, upper r2 or lower rmse) in the output of optimization and get better values of optimal net revenue